In [1]:
import pandas as pd
import numpy as np
import math
import datetime
from scipy import stats
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

Improvement:
1. Buat monthly rebalancing
2. Monte Carlo random period
3. Tambah parameter dgn sistem ranking (tanpa melalui create_database)

Factors yg didapat so far:
1. Size
2. Value
3. Momentum
4. Illiquidity
5. Price/Profitability

Use Python 3.9.15

In [2]:
pwd

'/Users/irfanhilman/factor_analysis'

In [30]:
database_1 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 1.csv')
database_2 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 2.csv')
database_3 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 3.csv')
database_4 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 4.csv')
database_5 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 5.csv')
database_6 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 6.csv')
database_7 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 7.csv')
database_8 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 8.csv')
database_9 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 9.csv')
database_10 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 10.csv')
database_11 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 11.csv')
database_12 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 12.csv')
database_13 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 13.csv')
database_14 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 14.csv')
database_15 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 15.csv')
database_16 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 16.csv')
database_17 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 17.csv')
database_18 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 18.csv')
database_19 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 19.csv')
database_20 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 20.csv')
database_21 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 21.csv')
database_22 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 22.csv')
database_23 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 23.csv')
database_24 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 24.csv')
database_25 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 25.csv')
database_26 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 26.csv')
database_27 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 27.csv')
database_28 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 28.csv')
database_29 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 29.csv')
database_30 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 30.csv')
database = pd.concat([database_1,database_2,database_3,database_4,database_5,database_6,database_7,database_8,database_9,database_10,
                      database_11,database_12,database_13,database_14,database_15,database_16,database_17,database_18,database_19,database_20,
                      database_21,database_22,database_23,database_24,database_25,database_26,database_27,database_28,database_29,database_30])
database = database.drop(columns='Unnamed: 0')
#Substitusi nilai Open, High, Low Price yg 0 dengan Close Price
database['Open Price'] = np.where(database['Open Price'].eq(0),database['Close Price'],database['Open Price'])
database['High Price'] = np.where(database['High Price'].eq(0),database['Close Price'],database['High Price'])
database['Low Price'] = np.where(database['Low Price'].eq(0),database['Close Price'],database['Low Price'])

In [4]:
database.columns

Index(['Kode', 'Date', 'Open Price', 'High Price', 'Low Price', 'Close Price',
       'Volume', 'Transaction Value', 'foreignsell', 'foreignbuy', 'Quarter',
       'Year', 'Quarter_Year', 'Total Pendapatan', 'Laba Kotor', 'Laba Usaha',
       'Laba Sebelum Pajak', 'Laba Bersih Tahun Berjalan', 'Saham Beredar',
       'Last Outstanding Shares', 'Total Aset Lancar',
       'Total Aset Tidak Lancar', 'Total Aset',
       'Total Liabilitas Jangka Pendek', 'Total Liabilitas Jangka Panjang',
       'Total Liabilitas', 'Total Ekuitas',
       'Total Arus Kas Dari Aktivitas Operasi',
       'Total Arus Kas Dari Aktivitas Investasi',
       'Total Arus Kas Dari Aktivitas Pendanaan', 'First Date', 'Age', 'IHSG',
       'Market Cap', 'Median Transaction Value 1 Month',
       'Momentum 252 Days Skip 0 Days', 'Momentum 21 Days Skip 0 Days',
       'Sales/Market', 'Earning/Market', 'PE Ratio', 'Previous Close Price'],
      dtype='object')

In [31]:
#BASIC PARAMETER
start_date = '2010-01-01'
end_date = '2025-03-21' #'2025-03-21'
hold_period = 21 #in days
factor = 'Sales/Market'
weighting = 'Value-Weight' #'Value-Weight','Equal-Weight'
hold_type = 'Periodic' #'Periodic' or 'First-Day in Month'
partition = 10 #Partition menentukan jumlah portfolio sesuai quartil. Contoh: partition 5 berarti satu portfolio mengandung 20% saham pada satu waktu



In [32]:
#SCREENING PARAMETER
screening_param = ['Market Cap','Total Ekuitas','Close Price','Median Transaction Value 1 Month'] #Params
screening_sign = ['Above','Above','Above','Above'] #Above, Below, Not Equal, or Between. If Between, assign a list in screening_val: screening_val = [[min,max]]
screening_val = [100_000_000_000,0,50,100_000_000] #Numeric variable


In [33]:
#Drop Stocks
drop_list = ['ihsg','excl']

In [34]:
database_date_list = list(database['Date'].sort_values().unique())          
if hold_type=='Periodic': 
    selected_date_list = database_date_list[::hold_period+1]
elif hold_type=='First-Day in Month':
    selected_date_list = []
    database['Year'] = database['Date'].str[:4].astype('int')
    database['Month'] = database['Date'].str[5:7].astype('int')
    for i in database['Year'].unique():
        database_i = database[database['Year']==i]
        for j in database_i['Month'].unique():
            database_j = database_i[database_i['Month']==j]
            database_j = database_j.sort_values(by='Date')
            date_list_j = list(database_j['Date'].unique())
            first_day = date_list_j[0]
            selected_date_list.append(first_day)
    database = database.drop(columns=['Year','Month'])
else:
    raise TypeError('hold_type wrong. Select "Periodic" or "First-Day in Month"')
buy_date_list = [d for d in selected_date_list if d >= start_date and d <= end_date]
sell_date_list = []
for i in buy_date_list[1:]:
    sell_date_i = database_date_list.index(i)-1
    sell_date = database_date_list[sell_date_i]
    sell_date_list.append(sell_date)
param_date_index = [(database_date_list.index(p)-1) for p in buy_date_list]
param_date_list = [database_date_list[p] for p in param_date_index]
basic_param = ['Date','Kode','Market Cap',factor]
basic_param = list(set(basic_param))
extend_param = basic_param.copy()
extend_param.extend(screening_param)
extend_param = list(set(extend_param))
database_part = database[extend_param]  
n_quantile=list(range(partition))
reb_comp_dic = {}
for q in n_quantile:
    reb_comp_dic[f"reb_comp_{q}"] = []
for en,i in enumerate(param_date_list):
    database_i = database_part[database_part['Date']==i]
    if hold_type=='Periodic': 
        pass 
    elif hold_type=='First-Day in Month':
        pass
    else: raise TypeError('hold_type wrong. Select "Periodic" or "First-Day in Month"')
    database_i = database_i.dropna()
    if len(drop_list)>0:
        for r in drop_list:
            database_i = database_i[database_i['Kode']!=r]
    else:
        pass
    if len(screening_param) == len(screening_sign) == len(screening_val):
        for ens,j in enumerate(screening_param):  #Initial Screening
            if screening_sign[ens] == 'Above':
                database_i = database_i[database_i[j]>screening_val[ens]]
            elif screening_sign[ens] == 'Below':
                database_i = database_i[database_i[j]<screening_val[ens]]
            elif screening_sign[ens] == 'Not Equal':
                database_i = database_i[database_i[j]!=screening_val[ens]]
            elif screening_sign[ens] == 'Between':
                database_i = database_i[(database_i[j]>=screening_val[ens][0])&(database_i[j]<=screening_val[ens][1])]
            else:
                raise TypeError('screening_sign wrong. Select either Above or Below')
    else:
        raise Exception('Number of elements in screening parameter list are different')
    database_i = database_i[basic_param]
    database_i['Quintile'] = pd.qcut(database_i[factor], q=partition, labels=False)
    portfolio_dic = {}
    for q in n_quantile:
        portfolio_dic[f"portfolio_{q}"] = database_i[database_i['Quintile']==q]
    for k in portfolio_dic.keys():  
        portfolio_k = portfolio_dic[k]
        if weighting=='Equal-Weight':
            portfolio_k['Weight'] = 1/len(portfolio_k)
        elif weighting=='Value-Weight':
            portfolio_k['Weight'] = portfolio_k['Market Cap']/portfolio_k['Market Cap'].sum()
        else:
            raise TypeError('weighting wrong. See notes in the parameter setting section')
        portfolio_dic[k] = portfolio_k
    for q in n_quantile:
        portfolio_k = portfolio_dic[f"portfolio_{q}"]
        try:
            portfolio_k['From'] = buy_date_list[en]
            portfolio_k['To'] = sell_date_list[en]
            portfolio_k['Number'] = en + 1
            portfolio_dic[f"portfolio_{q}"] = portfolio_k
            reb_comp_dic[f"reb_comp_{q}"].append(portfolio_k)
        except IndexError:
            pass
for q in n_quantile:
    reb_comp_q = pd.concat(reb_comp_dic[f"reb_comp_{q}"])
    reb_comp_q = reb_comp_q[['Kode','Weight','From','To','Number','Quintile']]
    reb_comp_q = reb_comp_q.reset_index(drop=True)
    reb_comp_dic[f"reb_comp_{q}"] = reb_comp_q


Kode dibawah masih bisa di-improve lagi, karena hanya melibatkan BUY dan SELL

In [36]:
buy_sell_date_list = buy_date_list + sell_date_list #
database_price = database[['Kode','Date','Open Price','High Price','Low Price','Close Price']] 
database_price = database_price[database_price['Date'].isin(buy_sell_date_list)] #
database_price['Kode + Date'] = database_price['Kode'].astype('str') + ' ' + database_price['Date'].astype('str')
reb_comp_act_dic = {}
for q in n_quantile:
    reb_comp_act_dic[f"reb_comp_act_{q}"] = []
for q in n_quantile:
    reb_comp = reb_comp_dic[f'reb_comp_{q}']
    start_date = reb_comp['From'].sort_values().astype('str').values[0]
    end_date = reb_comp['To'].sort_values().astype('str').values[-1]
    for i in list(reb_comp['Number'].sort_values().unique()):
        reb_comp_i = reb_comp[reb_comp['Number']==i].reset_index(drop=True)
        stock_list_i = list(reb_comp_i['Kode'])
        if len(stock_list_i) > len(set(stock_list_i)):
            print('Stock list is not unique')
            break
        else:
            pass
        from_date = reb_comp_i['From'].astype('str').unique()[0]
        reb_comp_act_i = pd.DataFrame({'Kode':stock_list_i})
        reb_comp_act_i['Date'] = from_date 
        reb_comp_act_i['Kode + Date'] = reb_comp_act_i['Kode'].astype('str') + ' ' + reb_comp_act_i['Date'].astype('str')
        reb_comp_act_i = reb_comp_act_i.merge(database_price.drop(columns=['Kode','Date']),how='left',on='Kode + Date')
        reb_comp_act_i = reb_comp_act_i.drop(columns='Kode + Date')
        reb_comp_act_i['Action'] = 'BUY'
        reb_comp_act_i = reb_comp_act_i.sort_values(by='Kode').reset_index(drop=True)
        reb_comp_act_dic[f"reb_comp_act_{q}"].append(reb_comp_act_i)
        to_date = reb_comp_i['To'].astype('str').unique()[0]
        reb_comp_act_i = pd.DataFrame({'Kode':stock_list_i})
        reb_comp_act_i['Date'] = to_date 
        reb_comp_act_i['Kode + Date'] = reb_comp_act_i['Kode'].astype('str') + ' ' + reb_comp_act_i['Date'].astype('str')
        reb_comp_act_i = reb_comp_act_i.merge(database_price.drop(columns=['Kode','Date']),how='left',on='Kode + Date')
        reb_comp_act_i = reb_comp_act_i.drop(columns='Kode + Date')
        reb_comp_act_i['Action'] = 'SELL'
        reb_comp_act_i = reb_comp_act_i.sort_values(by='Kode').reset_index(drop=True)
        reb_comp_act_dic[f"reb_comp_act_{q}"].append(reb_comp_act_i)
        #database_i = database_price[(database_price['Date']>=from_date)&(database_price['Date']<=to_date)]   #####
        #for d in pd.bdate_range(start=from_date,end=to_date):
        #    d = d.strftime('%Y-%m-%d') 
        #    if d in database_date_list:
        #        reb_comp_act_i = pd.DataFrame({'Kode':stock_list_i})
        #        reb_comp_act_i['Date'] = d
        #        reb_comp_act_i['Kode + Date'] = reb_comp_act_i['Kode'].astype('str') + ' ' + reb_comp_act_i['Date'].astype('str')
        #        reb_comp_act_i = reb_comp_act_i.merge(database_i.drop(columns=['Kode','Date']),how='left',on='Kode + Date')
        #        reb_comp_act_i = reb_comp_act_i.drop(columns='Kode + Date')
        #    else:
        #        continue
        #    if d==from_date:
        #        reb_comp_act_i['Action'] = 'BUY'
        #        reb_comp_act_i = reb_comp_act_i.sort_values(by='Kode').reset_index(drop=True)
        #        reb_comp_act_dic[f"reb_comp_act_{q}"].append(reb_comp_act_i)
        #    elif d==to_date:
        #        reb_comp_act_i['Action'] = 'SELL'
        #        reb_comp_act_i = reb_comp_act_i.sort_values(by='Kode').reset_index(drop=True)
        #       reb_comp_act_dic[f"reb_comp_act_{q}"].append(reb_comp_act_i)
        #   else:
        #        pass
for q in n_quantile:
    reb_comp_act_q = pd.concat(reb_comp_act_dic[f"reb_comp_act_{q}"])
    reb_comp_act_q = reb_comp_act_q.reset_index(drop=True)
    reb_comp_act_dic[f"reb_comp_act_{q}"] = reb_comp_act_q
database_price = database_price.drop(columns='Kode + Date')

In [ ]:
transaction_dic = {}
for q in n_quantile:
    transaction_dic[f"transaction_{q}"] = []
for q in n_quantile:
    reb_comp_act = reb_comp_act_dic[f"reb_comp_act_{q}"]
    reb_comp = reb_comp_dic[f'reb_comp_{q}']
    day_number = 0
    reb_comp_act_full_day_list = []
    for i in list(reb_comp_act['Date'].sort_values().unique()):
        reb_comp_act_i = reb_comp_act[reb_comp_act['Date']==i]
        reb_comp_act_i['Day Number'] = day_number
        reb_comp_i = reb_comp[reb_comp['From']==i]
        reb_comp_i = reb_comp_i.rename(columns={'From':'Date'})
        reb_comp_i['Kode + Date'] = reb_comp_i['Kode'].astype('str') + ' ' + reb_comp_i['Date'].astype('str') 
        reb_comp_act_s_list = []
        reb_comp_act_s_buy_list = []
        for s in list(reb_comp_act_i['Kode'].sort_values()):
            reb_comp_act_s = reb_comp_act_i[reb_comp_act_i['Kode']==s]
            reb_comp_act_s['Initial Capital'] = 1_000_000_000
            reb_comp_act_s['Reference Price'] = reb_comp_act_s['Close Price']
            reb_comp_act_s['Price/Lot'] = reb_comp_act_s['Reference Price'].astype('float64')*100
            reb_comp_act_s['Kode + Date'] = reb_comp_act_s['Kode'].astype('str') + ' ' + reb_comp_act_s['Date'].astype('str') 
            if reb_comp_act_s['Action'].values[0] == 'BUY':
                if day_number == 0:
                    reb_comp_act_s['Reference Capital'] = 1_000_000_000
                else:
                    reb_comp_act_s['Reference Capital'] = reb_comp_act_full_day['Total Capital'].iloc[-1]
                reb_comp_act_s = reb_comp_act_s.merge(reb_comp_i[['Kode + Date','Weight']],on='Kode + Date',how='left')
                reb_comp_act_s['Buy Price'] = reb_comp_act_s['Price/Lot']
                reb_comp_act_s['Sell Price'] = np.nan
                reb_comp_act_s['Buy Lot'] = reb_comp_act_s['Reference Capital']*reb_comp_act_s['Weight']/reb_comp_act_s['Buy Price']
                reb_comp_act_s['Sell Lot'] = np.nan
                reb_comp_act_s['Hold Lot'] = np.nan
                reb_comp_act_s['Capital Allocation'] = reb_comp_act_s['Reference Capital']*reb_comp_act_s['Weight']
                reb_comp_act_s['Invested'] = reb_comp_act_s['Buy Price']*reb_comp_act_s['Buy Lot']
                reb_comp_act_s['Sold'] = np.nan
                reb_comp_act_s['Market Value'] = reb_comp_act_s['Close Price']*reb_comp_act_s['Buy Lot']*100
                reb_comp_act_s['Potential P&L'] = reb_comp_act_s['Market Value'] - reb_comp_act_s['Invested']
                reb_comp_act_s['Realized P&L'] = np.nan
                reb_comp_act_s['Cash'] = reb_comp_act_s['Capital Allocation'] - reb_comp_act_s['Invested']
                reb_comp_act_s['ROI'] = np.nan
                reb_comp_act_s_buy_list.append(reb_comp_act_s)
            elif reb_comp_act_s['Action'].values[0] == 'SELL':
                reb_comp_act_full_day_s = reb_comp_act_full_day[reb_comp_act_full_day['Kode']==s]
                reb_comp_act_s['Reference Capital'] = reb_comp_act_full_day['Total Market Value'].iloc[-1]
                reb_comp_act_s['Reference Price'] = reb_comp_act_s['Close Price']
                reb_comp_act_s['Weight'] = 0
                reb_comp_act_s['Buy Price'] = np.nan
                reb_comp_act_s['Sell Price'] = reb_comp_act_s['Price/Lot']
                reb_comp_act_s = reb_comp_act_s.merge(reb_comp_act_buy[['Kode','Buy Lot']],on='Kode',how='left')
                reb_comp_act_s = reb_comp_act_s.rename(columns={'Buy Lot':'Sell Lot'})
                reb_comp_act_s['Buy Lot'] = np.nan
                reb_comp_act_s['Hold Lot'] = np.nan
                reb_comp_act_s['Capital Allocation'] = np.nan
                reb_comp_act_s = reb_comp_act_s.merge(reb_comp_act_full_day_s[['Kode','Invested']],on='Kode',how='left')
                reb_comp_act_s['Sold'] = reb_comp_act_s['Sell Price']*reb_comp_act_s['Sell Lot']
                reb_comp_act_s['Market Value'] = np.nan
                reb_comp_act_s['Potential P&L'] = np.nan
                reb_comp_act_s['Realized P&L'] = reb_comp_act_s['Sold']-reb_comp_act_s['Invested']
                reb_comp_act_s = reb_comp_act_s.merge(reb_comp_act_full_day_s[['Kode','Cash']],on='Kode',how='left')
                reb_comp_act_s['Cash'] = reb_comp_act_s['Cash'] + reb_comp_act_s['Sold']
                sold = reb_comp_act_s['Sold'].iloc[0]
                invested = reb_comp_act_buy[reb_comp_act_buy['Kode']==s]['Invested'].iloc[0]
                reb_comp_act_s['ROI'] = (sold-invested)/invested
            else: pass
            reb_comp_act_s_list.append(reb_comp_act_s)
        reb_comp_act_full_day = pd.concat(reb_comp_act_s_list)
        reb_comp_act_full_day['Total Invested'] = reb_comp_act_full_day['Invested'].sum()
        reb_comp_act_full_day['Total Market Value'] = reb_comp_act_full_day['Market Value'].sum()
        reb_comp_act_full_day['Total Potential P&L'] = reb_comp_act_full_day['Potential P&L'].sum()
        reb_comp_act_full_day['Total Realized P&L'] = reb_comp_act_full_day['Realized P&L'].sum()
        reb_comp_act_full_day['Total Cash'] = reb_comp_act_full_day['Cash'].sum() 
        reb_comp_act_full_day['Total Capital'] = reb_comp_act_full_day['Total Market Value'] + reb_comp_act_full_day['Total Cash']
        reb_comp_act_full_day['Weight'] = reb_comp_act_full_day['Weight'].fillna(value=reb_comp_act_full_day['Market Value']/reb_comp_act_full_day['Total Market Value'])
        day_number = day_number + 1
        transaction_dic[f"transaction_{q}"].append(reb_comp_act_full_day)
        if len(reb_comp_act_s_buy_list)>0:
            reb_comp_act_buy = pd.concat(reb_comp_act_s_buy_list)
for q in n_quantile:
    transaction_q = pd.concat(transaction_dic[f"transaction_{q}"])
    transaction_q = transaction_q.reset_index(drop=True)
    transaction_dic[f"transaction_{q}"] = transaction_q


In [11]:
hist_capital_dic = {}
for q in n_quantile:
    hist_capital_dic[f"hist_capital_{q}"] = []
for q in n_quantile:
    transaction_q = transaction_dic[f"transaction_{q}"]
    transaction_buy = transaction_q[transaction_q['Action']=='BUY']
    transaction_buy = transaction_buy.sort_values(by='Date')
    for i in transaction_buy['Date'].unique():
        transaction_buy_i = transaction_buy[transaction_buy['Date']==i]
        transaction_buy_i = transaction_buy_i[['Date','Total Capital']]
        transaction_buy_i = transaction_buy_i.head(1)
        hist_capital_dic[f"hist_capital_{q}"].append(transaction_buy_i)
for q in n_quantile:
    hist_capital_q = pd.concat(hist_capital_dic[f"hist_capital_{q}"])
    hist_capital_q['Return'] = hist_capital_q['Total Capital'].pct_change()
    hist_capital_q = hist_capital_q.reset_index(drop=True)
    hist_capital_dic[f"hist_capital_{q}"] = hist_capital_q
summary_dic = {}
for q in n_quantile:
    hist_capital_q = hist_capital_dic[f"hist_capital_{q}"]
    hist_capital_q = hist_capital_q.dropna()
    summary_dic[f"Average Portfolio Return Q{q}"] = hist_capital_q['Return'].mean()
first_q = n_quantile[0]
last_q = n_quantile[-1]
t_stat, p_value = stats.ttest_ind(hist_capital_dic[f"hist_capital_{first_q}"]['Return'].dropna(),hist_capital_dic[f"hist_capital_{last_q}"]['Return'].dropna(), equal_var=False)
summary_dic[f'Average Portfolio Return Top-Bottom'] = abs(summary_dic[f"Average Portfolio Return Q{first_q}"] - summary_dic[f"Average Portfolio Return Q{last_q}"])
summary_dic['t-stat'] = t_stat
summary_dic['p-value'] = p_value
ihsg = database[['Date','IHSG']]
ihsg = ihsg.drop_duplicates(subset='Date')
ihsg_ret = hist_capital_q[['Date']].merge(ihsg,on='Date',how='left')
ihsg_ret['Return'] = ihsg_ret['IHSG'].pct_change()
ihsg_ret = ihsg_ret.dropna()
summary_dic['Average Market Return'] = ihsg_ret['Return'].mean()
factor = factor.replace('/','-to-')
pd.DataFrame([summary_dic]).to_excel(f'Factor Testing Results for {factor} with {weighting} methodology and {hold_type} rebalancing.xlsx')



In [12]:
pd.DataFrame([summary_dic])

,Average Portfolio Return Q0,Average Portfolio Return Q1,Average Portfolio Return Q2,Average Portfolio Return Q3,Average Portfolio Return Q4,Average Portfolio Return Q5,Average Portfolio Return Q6,Average Portfolio Return Q7,Average Portfolio Return Q8,Average Portfolio Return Q9,Average Portfolio Return Top-Bottom,t-stat,p-value,Average Market Return
0,-0.008404,0.001496,0.004071,0.008575,0.005369,0.005499,0.008485,0.010541,0.015298,0.020102,0.028506,-3.079125,0.00225,0.00648
